In [3]:
import numpy as np
import pandas as pd
import re
import datetime

In [ ]:
file_path = 'desafio_murano.xlsx'

In [4]:
def inputs(df):
    while True:   
        dre = input("Preencha o seu DRE: ")
        if not re.match("^[0-9]{9}$" , dre):
            print ("Erro! Preencha o DRE corretamente (9 dígitos numéricos)")
        else:
            dre = str(dre)
            break

    while True:
        nome = input("Preencha o seu Nome Completo: ").title()
        if not re.match("^[^0-9]+$", nome):
            print ("Erro! Preencha seu nome sem números")
        else:
            break
    while True:   
        curso = input("Preencha o seu Curso: ").title()
        if not re.match("^[^0-9]+$", curso):
            print ("Erro! Preencha seu curso sem números")
        else:
            break

    while True:   
        genero = input("Preencha o seu Gênero (Masculino/Feminino): ")
        genero = genero.capitalize()
        if genero not in ['Feminino', 'Masculino']:
            print ("Erro! Tenha certeza que digitou corretamente")
        else:
            break

    while True:   
        nasc = input("Preencha a sua Data de Nascimento (dd/mm/aaaa): ")
        if not re.match("^(3[01]|[12][0-9]|0[1-9])/(1[0-2]|0[1-9])/[0-9]{4}$" , nasc):
            print ("Erro! Preencha a data corretamente (dd/mm/aaaa)")
        else:
            break

    while True:   
        altura = input("Preencha a sua Altura (em metros): ")
        altura = float(str(altura).replace(',','.'))
        if altura > 2.6 or altura < 0:
            print ("Erro! Tenha certeza que digitou corretamente (ex: 1,84)")
        else:
            break

    while True:   
        peso = input("Preencha o seu Peso (em kilos): ")
        peso = float(str(peso).replace(',','.'))
        if peso > 350 or peso < 0:
            print ("Erro! Tenha certeza que digitou corretamente (ex: 84)")
        else:
            break

    while True:   
        cra = input("Preencha o seu CR acumulado (0,0-10,0): ")
        cra = float(str(cra).replace(',','.'))
        if cra > 10 or cra < 0:
            print ("Erro! Tenha certeza que digitou corretamente (ex: 8,4)")
        else:
            break

    while True:   
        creditos = input("Preencha a sua Quantidade de Créditos Obtidos: ")
        try:
            creditos = int(creditos)
        except:
            pass
        if not isinstance(creditos, int) or creditos < 0:
            print ("Erro! Tenha certeza que digitou corretamente (ex: 390)")
        else:
            break

    while True:   
        renda = input("Preencha a sua Renda Mensal : ")
        try:
            renda = int(renda)
        except:
            pass
        if not isinstance(renda, int) or renda < 0:
            print ("Erro! Tenha certeza que digitou corretamente, sem pontos ou virgulas (ex: 3000)")
        else:
            break
    aux = pd.DataFrame([[dre, curso, nome, genero,
                                      nasc, altura, peso, cra,
                                      creditos, renda]], columns = df.columns )
    return aux

In [ ]:
def criarExcel(file_path):
    df = pd.DataFrame(columns = 'dre curso nome genero data_de_nascimento altura peso cra creditos_obtidos renda'.split())
    df.to_excel(file_path, index = False)
    return df

In [ ]:
def cadastrarAlunos(file_path):
    try:
        df = pd.read_excel(file_path)
    except:
        df = criarExcel(file_path)
        
    df = df.append(inputs(df))
    df.reset_index(drop = True, inplace = True )
    df.to_excel(file_path, index = False)
    return df

In [ ]:
def calculateAge(born):
    today = date.today()
    born = datetime.datetime.strptime(born, '%d/%m/%Y')
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [ ]:
def statistic(numeric, func, **kwargs ):
    df = pd.read_excel(file_path)
    df2 = df.copy()
    df2['idade'] = df.data_de_nascimento.apply(lambda x: calculateAge(x))
    
    keys = kwargs.keys()
    bool_list = [df2[key] == kwargs.get(key) for key in keys]
    
    df_slice = pd.Series(np.ones((len(df2)), dtype=bool))
    for i in bool_list:
        df_slice = i & df_slice

    return round(getattr(df2[df_slice][numeric], func)(),1)

In [ ]:
def printStatistics():
    
    print('A média de idade dos alunos é: ', end='') 
    print(statistic('idade','mean'))
    
    qtd_mul = statistic('dre','count', genero='Feminino')
    qtd_hom = statistic('dre','count', genero='Masculino')
    print(f'Temos {qtd_mul} alunas e {qtd_hom} alunos', end ='\n\n')

    print('A média de CRA dos homens que fazem engenharia é: ', end='') 
    print(statistic('cra','mean', genero='Masculino', curso='Engenharia'))

    print('A média de CRA das mulheres que fazem engenharia é: ', end = '') 
    print(statistic('cra','mean', genero='Feminino', curso='Engenharia'), end ='\n\n')

    print('A média de CRA das homens que fazem medicina é: ', end ='') 
    print(statistic('cra','mean', genero='Masculino', curso='Medicina'))
    
    print('A média de CRA das mulheres que fazem medicina é: ', end ='') 
    print(statistic('cra','mean', genero='Feminino', curso='Medicina'), end ='\n\n')

    print('A média de renda da medicina é: ', end ='') 
    print(statistic('renda','mean', curso='Medicina'), end = ', ')
    print('com desvio padrão de', end =' ')
    print(statistic('renda','std', curso='Medicina'))

    print('A média de renda da engenharia é: ', end ='') 
    print(statistic('renda','mean', curso='Engenharia'), end = ', ')
    print('com desvio padrão de', end =' ')
    print(statistic('renda','std', curso='Engenharia'))